# Groupby phase 01

## Create rows for the final table and groupby results

Attention: In the third block of code we must set the time window

In [1]:
import pandas as pd
import gc
import gzip
import os
from datetime import datetime, timedelta

# Set the display option to show all columns
pd.set_option('display.max_columns', None)

# Total file 
total_path = 'CSV\Exports\o10_eicu_four_concat_dataframes.csv'

# Read the CSV file
total_df = pd.read_csv(total_path)

# Copy the total_df in order to create the rows for the final table
temp_df = total_df[['uniquepid', 'patientunitstayid']].copy()

# Keep only unique values in the 'patientunitstayid' column
temp_df = temp_df.drop_duplicates(subset='patientunitstayid', keep='first')

# Sort the DataFrame by multiple columns
total_df = total_df.sort_values(by=['patientunitstayid', 'uniquepid', 'offset'])

C:\Users\dimop\AppData\Local\Temp\ipykernel_7436\1768891953.py:14: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  total_df = pd.read_csv(total_path)


In [2]:
"""
Creating a new column with the name charttime 
in order to groupby the result values based on
time. 
"""

# Convert 'offset' to timedelta and add it to the datetime threshold
threshold_datetime = datetime(2130, 1, 1, 0, 0, 0)
total_df['charttime'] = threshold_datetime + pd.to_timedelta(total_df['offset'], unit='m')

# Convert charttime column from object to datetime
total_df['charttime']= pd.to_datetime(total_df['charttime'])

In [3]:
# Initialize time window (start, increase and end)
start_increase_time = 3
end_time = 48

In [4]:
"""
Putting a new column at the end of the temp_df
which are going to use it as an index for the time
window.
"""

# Repeat each row based on the end_time/start_increase_time calculation
temp_df = temp_df.loc[temp_df.index.repeat(end_time/start_increase_time)].reset_index(drop=True)

# Create the "Time_Zone" column and count the occurrences of subject_id
temp_df['Time_Zone'] = temp_df.groupby('patientunitstayid').cumcount() + 1

# Save the resulting DataFrame to a new CSV file
temp_df.to_csv('CSV\Exports\o09_table_rows.csv', index=False)

# Mean Calculation

In [5]:
# Initialize the starting time window
time_window = pd.Timedelta(hours=start_increase_time)

# Define the desired time window
desired_time_window = pd.Timedelta(hours=end_time)

# Initialize a counter to track iterations
iteration = 1

# Define the directory path for saving files
output_directory = 'CSV/Exports/groupby/mean/'

# Create the directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# Convert 'result' column to numeric, handling non-convertible values with errors='coerce'
total_df['result'] = pd.to_numeric(total_df['result'], errors='coerce')

# Loop through the time windows
while time_window <= desired_time_window:
    grouped_multiple = (total_df.groupby(['patientunitstayid',
                                          'uniquepid',
                                          'observation',
                                          pd.Grouper(key='charttime',
                                                     freq=time_window)])
                        .agg({'result': ['mean']}))
    grouped_multiple.columns = ['Mean_Chart']
    grouped_multiple = grouped_multiple.reset_index()
    
    # Add a new column that counts from 1 for each unique combination
    grouped_multiple['Count'] = grouped_multiple.groupby(['patientunitstayid', 'uniquepid', 'observation']).cumcount() + 1
    
    # Add a 'File_Number' column with the current iteration value
    grouped_multiple['Time_Zone'] = iteration
    
    # Define the file path with forward slashes
    result_file_name = os.path.join(output_directory, f'results_iteration_mean_o{iteration}.csv')
    grouped_multiple.to_csv(result_file_name, index=False)
    
    # Increase the time window by 3 hours
    time_window += pd.Timedelta(hours=start_increase_time)
    
    # Increment the iteration counter
    iteration += 1

# Median Calculation

In [6]:
# Initialize the starting time window
time_window = pd.Timedelta(hours=start_increase_time)

# Define the desired time window
desired_time_window = pd.Timedelta(hours=end_time)

# Initialize a counter to track iterations
iteration = 1

# Define the directory path for saving files
output_directory = 'CSV/Exports/groupby/median/'

# Create the directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# Convert 'result' column to numeric, handling non-convertible values with errors='coerce'
total_df['result'] = pd.to_numeric(total_df['result'], errors='coerce')

# Loop through the time windows
while time_window <= desired_time_window:
    grouped_multiple = (total_df.groupby(['patientunitstayid',
                                          'uniquepid',
                                          'observation',
                                          pd.Grouper(key='charttime',
                                                     freq=time_window)])
                        .agg({'result': ['median']}))
    grouped_multiple.columns = ['Median_Chart']
    grouped_multiple = grouped_multiple.reset_index()
    
    # Add a new column that counts from 1 for each unique combination
    grouped_multiple['Count'] = grouped_multiple.groupby(['patientunitstayid', 'uniquepid', 'observation']).cumcount() + 1
    
    # Add a 'File_Number' column with the current iteration value
    grouped_multiple['Time_Zone'] = iteration
    
    # Define the file path with forward slashes
    result_file_name = os.path.join(output_directory, f'results_iteration_median_o{iteration}.csv')
    grouped_multiple.to_csv(result_file_name, index=False)
    
    # Increase the time window by 3 hours
    time_window += pd.Timedelta(hours=start_increase_time)
    
    # Increment the iteration counter
    iteration += 1

# Min Calculation

In [7]:
# Initialize the starting time window
time_window = pd.Timedelta(hours=start_increase_time)

# Define the desired time window
desired_time_window = pd.Timedelta(hours=end_time)

# Initialize a counter to track iterations
iteration = 1

# Define the directory path for saving files
output_directory = 'CSV/Exports/groupby/min/'

# Create the directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# Convert 'result' column to numeric, handling non-convertible values with errors='coerce'
total_df['result'] = pd.to_numeric(total_df['result'], errors='coerce')

# Loop through the time windows
while time_window <= desired_time_window:
    grouped_multiple = (total_df.groupby(['patientunitstayid',
                                          'uniquepid',
                                          'observation',
                                          pd.Grouper(key='charttime',
                                                     freq=time_window)])
                        .agg({'result': ['min']}))
    grouped_multiple.columns = ['Min_Chart']
    grouped_multiple = grouped_multiple.reset_index()
    
    # Add a new column that counts from 1 for each unique combination
    grouped_multiple['Count'] = grouped_multiple.groupby(['patientunitstayid', 'uniquepid', 'observation']).cumcount() + 1
    
    # Add a 'File_Number' column with the current iteration value
    grouped_multiple['Time_Zone'] = iteration
    
    # Define the file path with forward slashes
    result_file_name = os.path.join(output_directory, f'results_iteration_min_o{iteration}.csv')
    grouped_multiple.to_csv(result_file_name, index=False)
    
    # Increase the time window by 3 hours
    time_window += pd.Timedelta(hours=start_increase_time)
    
    # Increment the iteration counter
    iteration += 1

# Max Calculation

In [8]:
# Initialize the starting time window
time_window = pd.Timedelta(hours=start_increase_time)

# Define the desired time window
desired_time_window = pd.Timedelta(hours=end_time)

# Initialize a counter to track iterations
iteration = 1

# Define the directory path for saving files
output_directory = 'CSV/Exports/groupby/max/'

# Create the directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# Convert 'result' column to numeric, handling non-convertible values with errors='coerce'
total_df['result'] = pd.to_numeric(total_df['result'], errors='coerce')

# Loop through the time windows
while time_window <= desired_time_window:
    grouped_multiple = (total_df.groupby(['patientunitstayid',
                                          'uniquepid',
                                          'observation',
                                          pd.Grouper(key='charttime',
                                                     freq=time_window)])
                        .agg({'result': ['max']}))
    grouped_multiple.columns = ['Max_Chart']
    grouped_multiple = grouped_multiple.reset_index()
    
    # Add a new column that counts from 1 for each unique combination
    grouped_multiple['Count'] = grouped_multiple.groupby(['patientunitstayid', 'uniquepid', 'observation']).cumcount() + 1
    
    # Add a 'File_Number' column with the current iteration value
    grouped_multiple['Time_Zone'] = iteration
    
    # Define the file path with forward slashes
    result_file_name = os.path.join(output_directory, f'results_iteration_max_o{iteration}.csv')
    grouped_multiple.to_csv(result_file_name, index=False)
    
    # Increase the time window by 3 hours
    time_window += pd.Timedelta(hours=start_increase_time)
    
    # Increment the iteration counter
    iteration += 1

In [9]:
# Free Ram
total_df = None
temp_df = None
grouped_multiple = None

gc.collect()

0